##### Duration: ???

In [1]:
### Needs to module load cuda/11.7

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


In [4]:
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Python interpreter binary location: /home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


### Get recording info (google sheets)

In [5]:
import requests
import pandas as pd
from io import BytesIO

In [6]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [7]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10
1,M04003,30,24-05-01-13-51-17-581428,2024-05-01_13-51-09,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10
2,M04002,30,24-05-01-15-37-07-757018,2024-05-01_15-36-57,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Run 2D prediction

In [8]:
for idx, recording_row in recording_df.iterrows():
    break

In [9]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [10]:
output_directory_2D = output_directory/ "2D_predictions-2" 

In [11]:
output_directory_2D.mkdir(parents=True, exist_ok=True)

In [12]:
video_directory = Path(recording_row.video_location_on_o2) / recording_row.video_recording_id

In [13]:
pose_checkpoint = "/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth"
pose_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py'
det_checkpoint = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth'
det_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/config.py'

In [14]:
inferencer = Inferencer2D(
    video_directory,
    output_directory_predictions = output_directory_2D / recording_row.video_recording_id,
    pose_estimator_config = pose_config,
    pose_estimator_checkpoint = pose_checkpoint,
    detector_config = det_config,
    detector_checkpoint = det_checkpoint,
    use_tensorrt=False,
    expected_video_length_frames = 90000,
    
)

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Using CUDA device: NVIDIA_GeForce_RTX_4070_Ti
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Using Tensorrt: False
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Init completed


In [15]:
import mmdet

In [16]:
mmdet.__version__

'3.3.0'

In [17]:
mmdet.__file__

'/home/peromoseq/anaconda3/envs/mmdeploy/lib/python3.8/site-packages/mmdet/__init__.py'

In [18]:
inferencer.run()

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Running 2D inference over videos
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Video processing incomplete, running
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Processing 12 videos


processing videos:   0%|          | 0/12 [00:00<?, ?it/s]

Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth
Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth


INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:models loaded (not tensorrt)


frames:   0%|          | 0/90000 [00:00<?, ?it/s]

Exception caught: 'GetBBoxCenterScale is not in the mmdet::transform registry. Please check whether the value of `GetBBoxCenterScale` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'


KeyError: 'GetBBoxCenterScale is not in the mmdet::transform registry. Please check whether the value of `GetBBoxCenterScale` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'